In [8]:
import pandas as pd
import tensorflow as tf

import autokeras as ak
import numpy as np

In [1]:
from utils import HouseholdDataLoader, TSF_Data

dataload = HouseholdDataLoader()
data = dataload.data_by_days

tsf = TSF_Data(data=data['Global_active_power'],
               input_width=21,
               output_width=1,
               train_ratio=0.8)

tsf.normalize_data(standardization_type=1)

2022-06-16 16:08:02.650145: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [13]:
print(tsf.data_train[0].shape, tsf.data_train[1].reshape(1015).shape)

(1015, 21, 1) (1015,)


In [44]:
tsf.X_train.to_numpy()[..., np.newaxis].shape

(1037, 1)

In [42]:
tsf.X_test.to_numpy()[..., np.newaxis].shape

(289, 1)

In [15]:
predict_from = 1
predict_until = 10
lookback = 3
clf = ak.TimeseriesForecaster(
    lookback=lookback,
    predict_from=predict_from,
    predict_until=predict_until,
    max_trials=10,
    objective="val_loss",
)

# Train the TimeSeriesForecaster with train data
clf.fit(
    x=tsf.data_train[0],
    y=tsf.data_train[1],
    validation_data=(tsf.data_valid[0], tsf.data_valid[1]),
    batch_size=32,
    epochs=10,
)
# Predict with the best model(includes original training data).
# predictions = clf.predict(tsf.X_test.to_numpy()[..., np.newaxis])
# print(predictions.shape)
# Evaluate the best model with testing data.
# print(clf.evaluate(tsf.X_valid.to_numpy()[..., np.newaxis], tsf.X_valid.to_numpy()))


ValueError: Expect the data to TimeseriesInput to have shape (batch_size, num_features), but got input shape [32, 21, 1].

In [12]:
# Predict with the best model(includes original training data).
predictions = clf.predict(tsf.X_test.to_numpy()[..., np.newaxis])
print(predictions.shape)
# Evaluate the best model with testing data.
print(clf.evaluate(tsf.X_valid.to_numpy()[..., np.newaxis], tsf.X_valid.to_numpy()))

ValueError: The prediction data requires the original training data to make predictions on subsequent data points

In [8]:
data_x

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH
0,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9
1,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7
2,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0
3,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0
4,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6
...,...,...,...,...,...,...,...,...,...,...,...,...
6544,1.7,1111.0,-200.0,7.6,884.0,310.0,751.0,81.0,1280.0,1025.0,13.1,66.2
6545,2.3,1206.0,-200.0,9.8,969.0,363.0,701.0,95.0,1365.0,1178.0,14.3,63.3
6546,2.1,1202.0,-200.0,9.5,958.0,302.0,732.0,92.0,1330.0,1181.0,16.0,58.1
6547,2.7,1261.0,-200.0,11.2,1022.0,424.0,635.0,113.0,1407.0,1250.0,17.3,53.9


In [14]:
data_y

0       0.7578
1       0.7255
2       0.7502
3       0.7867
4       0.7888
         ...  
6544    0.9933
6545    1.0266
6546    1.0482
6547    1.0562
6548    1.0407
Name: AH, Length: 6549, dtype: float64

In [5]:
data_x_val

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH
6549,1.2,997.0,-200.0,4.6,742.0,179.0,911.0,82.0,1180.0,899.0,16.7,54.7
6550,1.4,1049.0,-200.0,5.5,790.0,221.0,870.0,96.0,1183.0,1019.0,15.4,58.9
6551,1.9,1082.0,-200.0,6.5,834.0,303.0,800.0,107.0,1207.0,1057.0,15.2,58.0
6552,2.6,1177.0,-200.0,10.3,987.0,429.0,692.0,119.0,1324.0,1229.0,16.2,54.5
6553,2.4,1166.0,-200.0,8.9,935.0,357.0,756.0,114.0,1301.0,1135.0,16.5,55.3
...,...,...,...,...,...,...,...,...,...,...,...,...
9352,3.1,1314.0,-200.0,13.5,1101.0,472.0,539.0,190.0,1374.0,1729.0,21.9,29.3
9353,2.4,1163.0,-200.0,11.4,1027.0,353.0,604.0,179.0,1264.0,1269.0,24.3,23.7
9354,2.4,1142.0,-200.0,12.4,1063.0,293.0,603.0,175.0,1241.0,1092.0,26.9,18.3
9355,2.1,1003.0,-200.0,9.5,961.0,235.0,702.0,156.0,1041.0,770.0,28.3,13.5


In [6]:
data_y_val

6549    1.0311
6550    1.0200
6551    0.9927
6552    0.9956
6553    1.0267
         ...  
9352    0.7568
9353    0.7119
9354    0.6406
9355    0.5139
9356    0.5028
Name: AH, Length: 2808, dtype: float64

In [47]:
predict_from = 1
predict_until = 10
lookback = 3
clf = ak.TimeseriesForecaster(
    lookback=lookback,
    predict_from=predict_from,
    predict_until=predict_until,
    max_trials=1,
    objective="val_loss",
)
# Train the TimeSeriesForecaster with train data
clf.fit(
    x=data_x,
    y=data_y,
    validation_data=(data_x_val, data_y_val),
    batch_size=32,
    epochs=10,
)
# Predict with the best model(includes original training data).
predictions = clf.predict(data_x_test)
print(predictions.shape)
# Evaluate the best model with testing data.
print(clf.evaluate(data_x_val, data_y_val))


INFO:tensorflow:Reloading Oracle from existing project ./time_series_forecaster/oracle.json


INFO:tensorflow:Reloading Oracle from existing project ./time_series_forecaster/oracle.json


INFO:tensorflow:Reloading Tuner from ./time_series_forecaster/tuner0.json


INFO:tensorflow:Reloading Tuner from ./time_series_forecaster/tuner0.json


INFO:tensorflow:Oracle triggered exit


...

INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


293/293 [==============================] - 3s 9ms/step
(10, 1)


Exception ignored in: <function _EagerDefinedFunctionDeleter.__del__ at 0x7f767cc68e50>
Traceback (most recent call last):
  File "/home/dspserver/anaconda3/envs/HO/lib/python3.9/site-packages/tensorflow/python/eager/function.py", line 303, in __del__
    try:
KeyboardInterrupt: 


88/88 [==============================] - 1s 6ms/step - loss: 2955.1204 - mean_squared_error: 2955.1204
[2955.120361328125, 2955.120361328125]


In [15]:
predictions

array([[0.8488437 ],
       [0.8489467 ],
       [1.0149212 ],
       [1.0149212 ],
       [1.014921  ],
       [1.0145173 ],
       [0.8488437 ],
       [1.4335226 ],
       [0.41878566],
       [0.70934165]], dtype=float32)